In [ ]:
from google.colab import userdata
import os

# 코랩 비밀번호 설정에 저장된 키를 가져옵니다.
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
%pip install -qU langchain_openai

# OpenAIEmbeddings
대표적인 사전 학습된 언어 모델: BERT와 GPT

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-large??" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

In [ ]:
# 텍스트를 임베딩하여 쿼리 결과를 생성합니다. -> 리스트
query_result = embeddings.embed_query(text)

In [ ]:
# 쿼리 결과의 처음 5개 항목을 선택합니다.
query_result[:5]

In [ ]:
doc_result = embeddings.embed_documents(
    [text]
)  # 텍스트를 임베딩하여 문서 벡터를 생성합니다.


In [ ]:
# 문서 결과의 첫 번째 요소에서 처음 5개 항목을 선택합니다.
doc_result[0][:5]


차원(dimensions) 조정

In [ ]:
# 문서 결과의 첫 번째 요소의 길이를 반환합니다.
len(doc_result[0])

In [ ]:
# 1536 차원-> 1024 차원으로 축소

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
# 주어진 텍스트를 임베딩하고 첫 번째 임베딩 벡터의 길이를 반환합니다.
len(embeddings_1024.embed_documents([text])[0])

유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [ ]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

# 캐시 임베딩

In [ ]:
pip install -r https://raw.githubusercontent.com/teddylee777/langchain-kr/main/requirements-mini.txt

In [ ]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [ ]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()
# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [ ]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

# 허깅페이스 임베딩

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from langchain_core.documents import Document

texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]


## HuggingFace Endpoint Embedding

In [ ]:
%pip install -qU \
    langchain-huggingface \
    langchain-community \
    huggingface_hub

In [ ]:
from google.colab import userdata
import os

# 코랩 비밀번호 설정에 저장된 Hugging Face API 키를 가져옵니다.
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
)

In [ ]:
%%time
# Document Embedding 수행
embedded_documents = hf_embeddings.embed_documents(texts)

In [ ]:
# Document Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [ ]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

#### query 와 embedding_document 간의 유사도 계산
이 코드는 **"벡터 검색 엔진이 내부적으로 어떤 수학적 원리로 작동하는가"**를 보여주기 위한 '원리 증명용' 코드입니다.

실제로 서비스를 개발할 때는 Chroma같은 '벡터 저장소(Vector Store)' 라이브러리를 사용

In [ ]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T
#@ 연산 (행렬 곱)

In [ ]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

In [ ]:
print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

### HuggingFace Embeddings

In [ ]:
%pip install -qU \
    transformers \
    peft \
    sentence-transformers

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"
# model_name = "intfloat/multilingual-e5-large"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)
%time
# Document
embedded_documents1 = hf_embeddings.embed_documents(texts)

### BGE-M3 임베딩